In [1]:
import os
from os.path import isdir, join
from pathlib import Path
import pandas as pd

# Math
import numpy as np
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
import librosa
import glob
import re
from sklearn.decomposition import PCA
import gc

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display as ipd
import librosa.display

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import pandas as pd

%matplotlib inline

In [2]:
train_audio_path = '../data/train/audio/'
filename = '/yes/0a7c2a8d_nohash_0.wav'
sample_rate, samples = wavfile.read(str(train_audio_path) + filename)

In [3]:
samples

array([ 14,  -7, -14, ...,  19,  20,  27], dtype=int16)

In [4]:
L = 16000
legal_labels = 'yes no up down left right on off stop go silence unknown'.split()

#src folders
root_path = r'..'
# out_path = r'.'
# model_path = r'.'
train_data_path = os.path.join(root_path, 'data', 'train', 'audio')
test_data_path = os.path.join(root_path, 'data', 'test', 'audio')

In [5]:
def list_wavs_fname(dirpath, ext='wav'):
    print(dirpath)
    fpaths = glob.glob(os.path.join(dirpath, r'*\\*' + ext))
    pat = r'.+\\(\w+)\\\w+\.' + ext + '$'
    labels = []
    for fpath in fpaths:
        r = re.match(pat, fpath)
        if r:
            labels.append(r.group(1))
    pat = r'.+\\(\w+\.' + ext + ')$'
    fnames = []
    for fpath in fpaths:
        r = re.match(pat, fpath)
        if r:
            fnames.append(r.group(1))
    return labels, fnames

In [6]:
labels, fnames = list_wavs_fname(train_data_path)


..\data\train\audio


In [7]:
def label_transform(labels):
    nlabels = []
    for label in labels:
        if label == '_background_noise_':
            nlabels.append('silence')
        elif label not in legal_labels:
            nlabels.append('unknown')
        else:
            nlabels.append(label)
    return pd.get_dummies(pd.Series(nlabels))

In [8]:
from tqdm import tqdm

In [25]:
new_sample_rate = 16000
y_train = []
x_train = []

for label, fname in tqdm(zip(labels, fnames)):
    sample_rate, samples = wavfile.read(os.path.join(train_data_path, label, fname))
#     samples = pad_audio(samples)
#     if len(samples) > 16000:
#         n_samples = chop_audio(samples)
    n_samples = [samples]
    for samples in n_samples:
        resampled = signal.resample(samples, int(new_sample_rate / sample_rate * samples.shape[0]))
#         _, _, specgram = log_specgram(resampled, sample_rate=new_sample_rate)
        y_train.append(label)
#         x_train.append(specgram)
x_train = np.array(x_train)
# x_train = x_train.reshape(tuple(list(x_train.shape) + [1]))
y_train = label_transform(y_train)
label_index = y_train.columns.values
y_train = y_train.values
y_train = np.array(y_train)
del labels, fnames
gc.collect()

64680it [02:11, 508.40it/s]C:\Users\agama\AppData\Local\Temp\ipykernel_15060\984529122.py:6: WavFileWarning:

Chunk (non-data) not understood, skipping it.

64727it [02:14, 480.98it/s]


9

In [28]:
x_train

array([], dtype=float64)